In [1]:
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
import smtplib
import os


In [2]:
smtp = smtplib.SMTP('smtp.gmail.com', 587)
smtp.ehlo()
smtp.starttls()
smtp.login('drowsyab2021@gmail.com', 'PASSWORD')


(235, b'2.7.0 Accepted')

In [4]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time


face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = load_model(r'C:\Users\SHARUN KUMAR\PycharmProjects\Drowsiness Detection\model\model.h5')

mixer.init()
sound = mixer.Sound(r'C:\Users\SHARUN KUMAR\PycharmProjects\Drowsiness Detection\alarm.wav')

cap = cv2.VideoCapture(0)
Score = 0
while True:
    ret, frame = cap.read()
    height, width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)

    cv2.rectangle(frame, (0, height - 50), (200, height), (0, 0, 0), thickness=cv2.FILLED)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, pt1=(x, y), pt2=(x + w, y + h), color=(255, 0, 0), thickness=3)

    for (ex, ey, ew, eh) in eyes:
        #cv2.rectangle(frame,pt1=(ex,ey),pt2=(ex+ew,ey+eh), color= (255,0,0), thickness=3 )

        # preprocessing steps
        eye = frame[ey:ey + eh, ex:ex + ew]
        eye = cv2.resize(eye, (80, 80))
        eye = eye / 255
        eye = eye.reshape(80, 80, 3)
        eye = np.expand_dims(eye, axis=0)
        # preprocessing is done now model prediction
        prediction = model.predict(eye)

        # if eyes are closed
        if prediction[0][0] > 0.10:
            cv2.putText(frame, 'closed', (10, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1,
                        color=(255, 255, 255),
                        thickness=1, lineType=cv2.LINE_AA)
            
            Score = Score + 1
            if (Score > 15):
                try:
                    sound.play()
                    import smtplib #importing the module
                    sender_add='drowsyab2021@gmail.com' #storing the sender's mail id
                    receiver_add='krishnarpana2020@gmail.com' #storing the receiver's mail id
                    password='Drowsy@124' #storing the password to log in
                     #creating the SMTP server object by giving SMPT server address and port number
                    smtp_server=smtplib.SMTP("smtp.gmail.com",587)
                    smtp_server.ehlo() #setting the ESMTP protocol
                    smtp_server.starttls() #setting up to TLS connection
                    smtp_server.ehlo() #calling the ehlo() again as encryption happens on calling startttls()
                    smtp_server.login(sender_add,password) #logging into out email id
                    msg_to_be_sent ='''
                    hello owner!!
                    Alert!!!!!!!
                    Driver is sleeping.......
                    '''
                    #sending the mail by specifying the from and to address and the message 
                    smtp_server.sendmail(sender_add,receiver_add,msg_to_be_sent)
                    print('Successfully the mail is sent') #priting a message on sending the mail
                    smtp_server.quit()#terminating the server


                    
                except:
                    pass

        # if eyes are open
        elif prediction[0][1] > 0.50: 
            cv2.putText(frame, 'open', (10, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1,
                        color=(255, 255, 255),
                        thickness=1, lineType=cv2.LINE_AA)
           
            if (Score < 0):
                Score = 0

    cv2.imshow('frame', frame)
    if cv2.waitKey(33) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
